In [ ]:
import numpy as np
import shap
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Load the dataset
breast = datasets.load_breast_cancer()
X = breast.data
y = breast.target

# Standardize the features for better performance
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE for oversampling the minority class
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Train the model
model = RandomForestClassifier()  # You can replace it with your preferred model
model.fit(X_train_smote, y_train_smote)

# Perform SHAP analysis
explainer = shap.Explainer(model, X_train_smote)
shap_values = explainer(X_test)

# Summarize the SHAP values
shap.summary_plot(shap_values, X_test, feature_names=breast.feature_names)

# Calculate and plot partial dependence
for feature in range(X_test.shape[1]):
    shap.dependence_plot(feature_names=breast.feature_names[feature],
                         shap_values=shap_values[:, feature],
                         features=X_test[:, feature],
                         interaction_index=None,
                         display_features=X_test,
                         title=f'Partial Dependence Plot for Feature: {breast.feature_names[feature]}')

# Evaluate the model on the test set
y_pred = model.predict(X_test)

# Print the classification report
print('Classification Report:')
print(classification_report(y_test, y_pred))
